In [251]:
from types import SimpleNamespace
import os
import simplejson as json

from datetime import datetime

def param_(data, parentKey='', level=0):
    rc = []    
    if isinstance(data, dict):
        rc.append(f'<tr><td style="text-align:right;font-size:90%;">{parentKey}</td><td colspan="2"></td></tr>')
        for key in data.keys():
            rc = rc + param_(data[key], key, level+1)
    else:
        if level == 1:
            rc.append(f'<tr><td style="text-align:right;font-size:90%;">{parentKey}</td><td colspan="2"></td></tr>')
            rc.append(f'<tr><td></td><td colspan="2"><small>{str(data)}</small></td></tr>')
        else:
            rc.append(f"<tr><td></td><td colspan='2'><small>{parentKey}={str(data)}</small></td></tr>")

    return rc


def json2html_(data):

    controller = SimpleNamespace(**data)

    if not hasattr(controller,"ctl"):
        return ""
        
    text = "<!DOCTYPE html>"
    text += "<html lang='en'>"

    text += "<head>"
    text += "<meta http-equiv='Content-Type' content='text/html; charset=iso-8859-1'>"
    text += "<style>"
    text += "body { font-family:verdana; padding:12px; }"

    text += ".arrow { border: solid #808080; border-width: 0 4px 4px 0; display: inline-block; padding: 0px; width:8px; height:8px;margin-top:6px; }"
    text += ".right { transform: rotate(-45deg); -webkit-transform: rotate(-45deg); }"
    text += ".left { transform: rotate(135deg); -webkit-transform: rotate(135deg); }"
    text += ".up { transform: rotate(-135deg); -webkit-transform: rotate(-135deg); }"
    text += ".down { transform: rotate(45deg); -webkit-transform: rotate(45deg); }"

    text += "#graph td { font-family:verdana; width: 32px;height: 32px; text-align:center; }"
    text += "#graph td.state { font-family:verdana; color:white; width: 32px; height: 32px; background: radial-gradient(ellipse at center, #c60000 0%,#c60000 70%,#c60000 70%,#404040 50%,#ffffff 50%); }"
    
    text += "#graph td.stop { font-family:verdana; color:white; width: 32px; height: 32px; background: radial-gradient(ellipse at center, #808080 0%,#808080 70%,#808080 70%,#404040 50%,#ffffff 50%); }"
    text += "#graph td.start { font-family:verdana; color:white; width: 32px; height: 32px; background: radial-gradient(ellipse at center, #a0a0a0 0%,#a0a0a0 70%,#a0a0a0 70%,#404040 50%,#ffffff 50%); }"
    
    text += "#graph td.transition { font-family:verdana; background-color:whitesmoke; }"
    text += "#graph td.cycles { font-family:verdana; color:#666666; font-size:90%; background-color:whitesmoke; }"
    text += "#graph td.space { font-family:verdana; color:white; background-color:transparent; }"

    text += "td.title { font-family:verdana; color:#c90000; font-weight:bold; text-align:center; background-color:whitesmoke;line-height:200%; }"
    text += "td.headline { font-family:verdana; color:#666666; font-weight:bold; }"

    text += "</style>"
    text += "</head>"
    
    text += "<body>"

    text += "<table border='0' cellpadding='2px'>"

    titles = ['left stroke', 'right stroke']

    items = controller.items
    ctl = SimpleNamespace(**controller.ctl)

    text += '<tr><td></td><td colspan="2" style="text-align:right;font-size:70%;">'
    text += 'id:' + ctl.id + 'key:' + ctl.key + ', author: ' + ctl.author + ', version: ' + ctl.version + ', continuous: <span style="font-weight:bold;">' + ('on' if ctl.continuous != -1 else 'off') + '</span>'
    text += '</td></tr>'
    text += '<tr><td></td><td colspan="2" style="text-align:right;font-size:70%;">'
    text += '' + str(datetime.now())
    text += '</td></tr>'

    text += f"<tr><td colspan='3' class='title'>{ctl.title}</td></tr>"

    #
    # graph
    #

    N = len(items.keys())

    text += "<tr><td colspan='3' class='headline'>graph</td></tr>"
    
    text += "<tr><td></td><td colspan='2'>"
    
    text += "<table id='graph' border='0'>"         
    text += "<tr>"
    text += "<td class='start'>&nbsp;</td><td class='transition'><div class='arrow right'>&nbsp;</div></td>"
    text += "<td class='state'>1</td><td class='transition'><div class='arrow right'>&nbsp;</div></td>"

    for row, key in enumerate(items.keys()):
        item = SimpleNamespace(**items[key])
        if item.goto:
            text += f"<td class='state'>{int(item.goto)+1}</td>"
            if N - 2 != row:
                text += "<td class='transition'><div class='arrow right'>&nbsp;</div></td>"
            else:
                if ctl.continuous == -1:
                    text += "<td class='transition'><div class='arrow right'>&nbsp;</div></td>"
                    text += "<td class='stop'>&nbsp;</td>"
                else:
                    text += "<td class='space'>&nbsp;</td>"
        else:
            if ctl.continuous != -1:

                text += "</tr><tr>"
                for c in range(ctl.continuous + 1):
                    text += "<td class='space'>&nbsp;</td><td class='space'>&nbsp;</td>"
                text += "<td class='transition'><div class='arrow up'>&nbsp;</div></td>"
                for c in range(ctl.continuous, row - 1):
                    text += "<td class='space'>&nbsp;</td><td class='space'>&nbsp;</td>"
                text += "<td class='space'>&nbsp;</td><td class='transition'><div class='arrow down'>&nbsp;</div></td></tr><tr>"

                for c in range(ctl.continuous + 1):
                    text += "<td class='space'>&nbsp;</td><td class='space'>&nbsp;</td>"
                
                text += "<td class='transition'>&nbsp;</td>"

                for c in range(ctl.continuous, row - 1):
                    text += "<td class='transition'><div class='arrow left'>&nbsp;</div></td><td class='transition'><div class='arrow left'>&nbsp;</div></td>"
                
                text += "<td class='transition'><div class='arrow left'>&nbsp;</div></td>"
                text += "<td class='cycles'>" + (f"{ctl.param['cycles']['setpoint']}x" if ctl.param and 'cycles' in ctl.param.keys() else "&nbsp;") + "</td>"

            else:
                text += "</tr><tr>" + "".join((2 * N * ["<td>&nbsp;</td>"]))
    
    text += "</tr></table></td></tr>"    

    #
    # flow
    #
        
    text += f"<tr><td colspan='3' class='headline'>flow</td></tr>"
    text += "<tr><td></td>"
    text += "<td><b>" + titles[0] + "</b></td><td><b>" + titles[1] + "</b></td>"
    text += "</tr>"
    
    for row, key in enumerate(items.keys()):

        item = SimpleNamespace(**items[key])
        
        text += "<tr style='background-color:whitesmoke;'><td>[" + str(row+1) + "]</td>"

        for col in [0,1]:
            text += "<td>" + item.name[col] + "</td>"

        text += "</tr>"
        
        text += "<tr><td></td>"
        
        for col in [0,1]:

            mode = item.mode[col]

            if mode == 'valve':

                text += '<td><small>'
                text += 'inject valve is ' + ('open' if item.valve[col][0] == 1 else 'closed')
                text += ', refill valve is ' + ('open' if item.valve[col][1] == 1 else 'closed')
                text += ', ' + ('waits on pair' if item.pending[col] else 'proceeds immediately')
                text += '</small></td>'

            elif mode == 'rate':

                text += '<td valign="top"><small>'
                text += 'with a rate of ' + str(item.value[col][0]) + 'µl/min,'
                
                if (item.value[col][1]):
                    if (item.domain[col][1] == "volume"):
                        text += ' until ' + str(item.value[col][1]) + 'ml reached, '
                    else:
                        text += ' for min. ' + str(item.value[col][1]) + 's, '
                
                text += 'inject valve is ' + ('open' if item.valve[col][0] == 1 else 'closed')
                text += ', refill valve is ' + ('open' if item.valve[col][1] == 1 else 'closed')

                text += ', ' + ('waits on pair' if item.pending[col] else 'proceeds immediately')

                text += '</small></td>'

            elif 'pressure' == item.mode[col]:
                                
                text += '<td valign="top"><small>'
                
                if (item.value[col][0]):
                    text += 'pressure up to ' + str(item.value[col][0]) + 'bar, '
                else:
                    text += 'hold pressure of the ' + titles[item.channel[col]] + ' until it\'s target volume is reached, '

                if (item.value[col][1]):
                    if (item.domain[col][1] == "volume"):
                        text += ' until ' + str(item.value[col][1]) + 'ml reached, '
                    else:
                        text += ' for min. ' + str(item.value[col][1]) + 's, '
                                
                text += 'inject valve is ' + ('open' if item.valve[col][0] == 1 else 'closed')
                text += ', refill valve is ' + ('open' if item.valve[col][1] == 1 else 'closed')

                text += ', ' + ('waits on pair' if item.pending[col] else 'proceeds immediately')

                text += '</small></td>'

        text += "</tr>"
            
        text += "<tr><td></td>"
        text += '<td colspan="2" style="text-align:right;font-size:70%;">'
        if item.goto:
            text += 'continue with step ' + item.goto
        else:
            if ctl.continuous == -1:
                text += 'stop with this step'
            else:
                text += 'continue with step ' + str(ctl.continuous + 1)
                text += f", {ctl.param['cycles']['setpoint']} cycles" if ctl.param and "cycles" in ctl.param.keys() else ""
        
        text += '</td>'

        text += "</tr>"

    #
    # parameter
    #

    text += "<tr><td colspan='3' class='headline'>parameter</td></tr>"
    if not ctl.param:
        text += "<tr><td colspan='3'>none</td></tr>"    
    else:        
        rc = param_(ctl.param)
        text += "".join(rc)

    #
    #
    #

    text += "</table>"

    text += "</body>"
    text += "</html>"

    return text

def load_(filename):
    rc = ''
    with open(fname, "r") as f:
        for line in f:
            rc += line.encode("latin-1").decode("utf-8")
    return json.loads(rc)


folder = os.path.join(os.getcwd(), "..", "lib")
fnames = [os.path.join(folder,fname) for fname in os.listdir(folder) if fname.endswith('.json')]

for fname in fnames[:]:
    html = json2html_(load_(fname))
    fname = fname + ".html"
    fname = os.path.join(folder, "doc", os.path.basename(fname))
    print(fname)
    with open(fname, "w") as f:
        f.write(html)


c:\Develop\py\FluidicsLab\Drafts\bus\books\..\lib\doc\2-test.json.html
c:\Develop\py\FluidicsLab\Drafts\bus\books\..\lib\doc\pressure.json.html
c:\Develop\py\FluidicsLab\Drafts\bus\books\..\lib\doc\profile.config.json.html
c:\Develop\py\FluidicsLab\Drafts\bus\books\..\lib\doc\purge.json.html
c:\Develop\py\FluidicsLab\Drafts\bus\books\..\lib\doc\rate.json.html
c:\Develop\py\FluidicsLab\Drafts\bus\books\..\lib\doc\refill.json.html
